In [11]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import scipy.sparse as sp
from scipy.sparse import coo_matrix
from lightfm import LightFM
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import StratifiedKFold

In [2]:
train = pd.read_pickle(FILE.train_final.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_final.value)
print('test shape is: {}'.format(test.shape))

X = pd.concat([train,test],sort=False)
print(X.shape)




print(X.shape)

train shape is: (2992639, 36)
test shape is: (80276, 35)
(3072915, 36)
(3072915, 36)


# Step 1, change data type

In [3]:
for col in X.columns:
    if X[col].dtype == 'float64':
        X[col] = X[col].astype(utils.set_type(X[col],'float'))
    if X[col].dtype == 'int64':
        X[col] = X[col].astype(utils.set_type(X[col],'int'))
        
print(sys.getsizeof(X)/1024**3)

2.351760576479137


# Step 4, drop those only has 1 unique value

In [4]:
drop_col = []
for col in X.columns:
    if X[col].nunique() == 1:
        drop_col.append(col)
        
print(drop_col)

X.drop(drop_col,axis=1,inplace=True)

['creative_is_js', 'creative_is_voicead', 'app_paid']


In [5]:
le = LabelEncoder()
not_categorical = ['instance_id','time', 'datetime','click']
categorical_cols = list(set(X.columns) - set(not_categorical))
print(len(categorical_cols))


for col in tqdm(categorical_cols):
    if X[col].dtype == 'object':
        X[col] = X[col].fillna('FillNAValue')
        X[col] = le.fit_transform(X[col])
    else:
        pass
#         X[col] = X[col].fillna(-100000)
#         X[col] = le.fit_transform(X[col])

  0%|          | 0/30 [00:00<?, ?it/s]

30


100%|██████████| 30/30 [00:27<00:00,  1.09it/s]


In [6]:


userid_col = {
'uid1':['model'],
'uid2':['model','city','province'],
'uid3':['model','make','os_name','osv','os']}

userraw_col = {
'uraw1':['app_id'],
'uraw2':['carrier','nnt'],
'uraw3':['app_cate_id','f_channel','app_id']}
item_col = {
'itemc1':['adid','creative_width','creative_height','inner_slot_id'],
'itemc2':['adid','orderid','campaign_id','creative_id','creative_type','creative_tp_dnf','creative_width','creative_height','inner_slot_id'],
'itemc3':['adid','advert_id','orderid','advert_industry_inner','advert_name','campaign_id','creative_id','creative_type','creative_tp_dnf','creative_width','creative_height']}


# Define Function List

In [7]:
def get_new_name(cob_col,func):
    return '+'.join(cob_col)+'_'+func.__name__

def matrix_factorization(df_history, df, target, item_col, userid_col, userraw_col):
    """
    userid_col is unique user id
    item_col is unique itme id
    userraw_col is used to construct user feature. dim: user_id*userraw
    """
    dff = pd.DataFrame()
    dff_history = pd.DataFrame()


    #1. process item
    if item_col is None:
        dff['item'] = np.zeros(len(df))
        dff_history['item'] = np.zeros(len(df_history))
    else:
        encoder = LabelEncoder()
        group = get_group(df, item_col)
        group_history = get_group(df_history, item_col)
        encoder.fit(pd.concat([group, group_history]))
        dff['item'] = encoder.transform(group)
        dff_history['item'] = encoder.transform(group_history)
#     print('processing item done!')

    #2. user raw
    group = get_group(df, userraw_col)
    group_history = get_group(df_history, userraw_col)
    encoder = LabelEncoder()
    encoder.fit(pd.concat([group, group_history]))
    dff['userraw'] = encoder.transform(group)
    dff_history['userraw'] = encoder.transform(group_history)
#     print('processing user raw done')


    #3. user_id
    group = get_group(df, userid_col)
    group_history = get_group(df_history, userid_col)
    encoder = LabelEncoder()
    encoder.fit(pd.concat([group, group_history]))
    dff['user_id'] = encoder.transform(group)
    dff_history['user_id'] = encoder.transform(group_history)
#     print('processing user id done')



    num_users = max(dff.user_id.max(), dff_history.user_id.max()) + 1
    num_items = max(dff.item.max(), dff_history.item.max()) + 1
    num_userraw = max(dff.userraw.max(), dff_history.userraw.max()) + 1

    M = coo_matrix(
            (df_history[target], ( dff_history.user_id, dff_history.item)),
            shape=(num_users, num_items)
        )

    user_features = pd.concat([dff, dff_history])[['userraw', 'user_id']].drop_duplicates()

    user_features = coo_matrix(
        (np.ones(len(user_features)), (user_features.user_id, user_features.userraw)),
        shape=(num_users, num_userraw)
    )

    user_features = sp.hstack([sp.eye(num_users), user_features])

    model = LightFM(no_components=50, learning_rate=0.1)
    print('fitting lightFM')
    model.fit(
            M, 
            epochs=2, 
            num_threads=18, 
            user_features=user_features,
        )
    print('predicting lightFM')
    result = model.predict(
        dff.user_id.values, 
        dff.item.values, 
        user_features=user_features,
    )
    return result




def get_group(df,cols):
    group = df[cols[0]].astype(str).copy()
    for col in cols[1:]:
        group += '_' + df[col].astype(str)
    return group

In [8]:
# ss = matrix_factorization(train,train,'click',
#                      item_col=ads_list[3],
#                      userid_col=['model'],userraw_col=user_list[0])

In [9]:


train_length = len(train)
train = X.iloc[:train_length].copy()
test = X.iloc[train_length:].copy()

In [19]:
n_fold = 5
seed = 19
skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
try:
    X_train = pd.read_pickle(FILE.X_fe_train_libfm.value)
    X_test = pd.read_pickle(FILE.X_fe_test_libfm.value)
except:
    print('no libfm feature found. create new one')
    X_train = pd.DataFrame()
    X_train['instance_id'] = train['instance_id'].values
    X_test = pd.DataFrame()
    X_test['instance_id'] = test['instance_id'].values
count = 0
for uid in userid_col:
    for uraw in userraw_col:
        for ic in item_col:
            test_list = []
            feature_name = get_new_name((uid,uraw,ic),matrix_factorization)
            print('####{}  {}'.format(count,feature_name))
            if feature_name in X_train.columns and feature_name in X_test.columns:
                continue
            X_train[feature_name] = np.nan
            X_test[feature_name] = np.nan
            for t,v in skf.split(train, train.click):
                history = train.loc[t].copy()
                val = train.loc[v].copy()
                df_now = pd.concat([val,test],sort=False)
                ppreds = matrix_factorization(history,df_now,target='click',
                                                     item_col=item_col[ic],
                                                     userid_col=userid_col[uid],
                                                     userraw_col=userraw_col[uraw])

                test_preds = ppreds[len(val):]
                test_list.append(test_preds)
                val_preds = ppreds[:len(val)]
                
#                 holdout_preds = matrix_factorization(history,holdout,target='click',
#                                                      item_col=item_col[ic],
#                                                      userid_col=userid_col[uid],
#                                                      userraw_col=userraw_col[uraw])
#                 holdout_preds_list.append(holdout_preds)
#                 test_preds = matrix_factorization(history,test,target='click',
#                                                      item_col=item_col[ic],
#                                                      userid_col=userid_col[uid],
#                                                      userraw_col=userraw_col[uraw])
#                 test_list.append(test_preds)
#                 val_preds = matrix_factorization(history,val,target='click',
#                                                      item_col=item_col[ic],
#                                                      userid_col=userid_col[uid],
#                                                      userraw_col=userraw_col[uraw])
                X_train.loc[v,feature_name] = val_preds
            test_list = np.array(test_list)
            test_preds_final = np.mean(test_list,axis=0)
            X_test[feature_name] = test_preds_final
            X_train.to_pickle(FILE.X_fe_train_libfm.value)
            X_test.to_pickle(FILE.X_fe_test_libfm.value)
            count+=1
                

####0  uid1+uraw1+itemc1_matrix_factorization
####0  uid1+uraw1+itemc2_matrix_factorization
####0  uid1+uraw1+itemc3_matrix_factorization
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
####1  uid1+uraw2+itemc1_matrix_factorization
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
####2  uid1+uraw2+itemc2_matrix_factorization
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
####3  uid1+uraw2+itemc3_matrix_factorization
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
fitting lightFM
predicting lightFM
####4  uid1+uraw3+itemc1

In [ ]:

try:
    X_train = pd.read_pickle(FILE.X_fe_train_libfm.value)
    X_test = pd.read_pickle(FILE.X_fe_test_libfm.value)
except:
    print('no libfm feature found. create new one')
    X_train = pd.DataFrame()
    X_train['instance_id'] = train['instance_id'].values
    X_test = pd.DataFrame()
    X_test['instance_id'] = test['instance_id'].values
count = 0
for uid in userid_col:
    for uraw in userraw_col:
        for ic in item_col:
            holdout_preds_list = []
            test_list = []
            feature_name = get_new_name((uid,uraw,ic),matrix_factorization)
            print('####{}  {}'.format(count,feature_name))
            if feature_name in X_train.columns and feature_name in X_test.columns:
                continue
            X_train[feature_name] = np.nan
            X_test[feature_name] = np.nan
            for fold,(v,t) in day_index.items():
                history = train.loc[t].copy()
                holdout = train.loc[holdout_index].copy()
                val = train.loc[v].copy()
                df_now = pd.concat([val,holdout,test],sort=False)
                ppreds = matrix_factorization(history,df_now,target='click',
                                                     item_col=item_col[ic],
                                                     userid_col=userid_col[uid],
                                                     userraw_col=userraw_col[uraw])
                holdout_preds = ppreds[len(val):len(val)+len(holdout)]
                holdout_preds_list.append(holdout_preds)
                test_preds = ppreds[len(val)+len(holdout):]
                test_list.append(test_preds)
                val_preds = ppreds[:len(val)]
                
#                 holdout_preds = matrix_factorization(history,holdout,target='click',
#                                                      item_col=item_col[ic],
#                                                      userid_col=userid_col[uid],
#                                                      userraw_col=userraw_col[uraw])
#                 holdout_preds_list.append(holdout_preds)
#                 test_preds = matrix_factorization(history,test,target='click',
#                                                      item_col=item_col[ic],
#                                                      userid_col=userid_col[uid],
#                                                      userraw_col=userraw_col[uraw])
#                 test_list.append(test_preds)
#                 val_preds = matrix_factorization(history,val,target='click',
#                                                      item_col=item_col[ic],
#                                                      userid_col=userid_col[uid],
#                                                      userraw_col=userraw_col[uraw])
                X_train.loc[v,feature_name] = val_preds
            holdout_preds_list = np.array(holdout_preds_list)
            test_list = np.array(test_list)
            holdout_preds_final = np.mean(holdout_preds_list,axis=0)
            test_preds_final = np.mean(test_list,axis=0)
            X_train.loc[holdout_index,feature_name] = holdout_preds_final
            X_test[feature_name] = test_preds_final
            X_train.to_pickle(FILE.X_fe_train_libfm.value)
            X_test.to_pickle(FILE.X_fe_test_libfm.value)
            count+=1
                

In [ ]:
history['click']